# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****eJgA
LANGSMITH_API_KEY=****2eed
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ject


In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-openai langgraph langchain-groq groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing ins

In [ ]:
import getpass
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## Human👨‍💻 and AI 🤖 Messages

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian"
)

In [ ]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [ ]:
print(result["messages"][-1].content)

Hey there! I’m feeling as refreshed as a freshly‑deployed microservice after a zero‑downtime rollout—no bugs, just jokes. How’s your stack today? 🚀😄


In [ ]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [ ]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m feeling as refreshed as a freshly‑deployed microservice after a zero‑downtime rollout—no bugs, just jokes. How’s your stack today? 🚀😄



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

In [ ]:
agent = create_agent(
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [ ]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond hush,  
Leather thuds—time arcs.  
Pitch whispers, batter answers,  
Grass stains the sunrise.  
Three outs, a season's sigh.


#### Dictionaries

In [ ]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Lightning on track—  
muscles fire, breath held tight,  
finish line whispers.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages.

In [ ]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [ ]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [ ]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Lightning on the field
Footsteps echo, hearts race fast
Victory whispers
checking haiku, it has 3 lines:
 Grass glistens, cleats strike
Crowd roars, the ball arcs high
Champions rise anew
checking haiku, it has 3 lines:
 Sweat glistens, ball flies
Net ripples, cheers surge like tide
Champions breathe triumph
checking haiku, it has 3 lines:
 Morning mist lifts, 
Runner's breath syncs with sunrise, 
Finish line glows gold
checking haiku, it has 3 lines:
 Stadium lights flicker
Leather spirals through night air
Victory hums low
checking haiku, it has 3 lines:
 Thunderous drums rise
Athletes sprint, breath in sync
Gold glints on the edge
checking haiku, it has 3 lines:
 Echoes on the court
Swoosh of net, swift feet in flight
Champions chase dawn
checking haiku, it has 3 lines:
 Whistle pierces dusk
Players surge, the ball ignites
Glory sparks the night
checking haiku, it has 3 lines:
 Leather thuds on grass
Crowd erupts, the goal alights
Victory breathes de

In [ ]:
result["messages"][-1].content

'Swift wind on the track  \nFeet pound rhythm, hearts alight—  \nFinish line glows gold'

In [ ]:
print(len(result["messages"]))

30


In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (fc_d1aa4cb2-56c6-4e23-880e-d8afa75aef4a)
 Call ID: fc_d1aa4cb2-56c6-4e23-880e-d8afa75aef4a
  Args:
    text: Lightning on the field
Footsteps echo, hearts race fast
Victory whispers
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (fc_d527b001-d6df-401e-846e-c014b192de1f)
 Call ID: fc_d527b001-d6df-401e-846e-c014b192de1f
  Args:
    text: Grass glistens, cleats strike
Crowd roars, the ball arcs high
Champions rise anew
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [ ]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='13b18ae4-f92e-4582-925a-39d7984806a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 555, 'prompt_tokens': 170, 'total_tokens': 725, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbI0V3W0uPCDlcnD4GYTydYxCsAJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0055426a-ee06-4daa-a36c-4074fa9c1ceb-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'}, 'id': 'call_uOLf0ZxcBTNBT2a77IZJi1jO', 'type': 'tool_call'}], usage_

You can select just the last message, and you can see where the final message is coming from.

In [ ]:
result["messages"][-1]

AIMessage(content='Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 231, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--61eb6a33-1bda-46d7-8d00-f3aaea2487a3-0', usage_metadata={'input_tokens': 231, 'output_tokens': 21, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
result["messages"][-1].usage_metadata

{'input_tokens': 231,
 'output_tokens': 21,
 'total_tokens': 252,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [ ]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [ ]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()